In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy.spatial.transform import Rotation as R
import time
import pickle
import argparse 
from utils import *

from deepVCP import DeepVCP
from ModelNet40Dataset import ModelNet40Dataset
from KITTIDataset import KITTIDataset
from deepVCP_loss import deepVCP_loss

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
  

In [2]:
dataset = "modelnet"
retrain_path = False
model_path = "final_model.pt"
full_dataset = True 

In [3]:
num_epochs = 10
batch_size = 2
lr = 0.001
# loss balancing factor 
alpha = 0.5

print(f"Params: epochs: {num_epochs}, batch: {batch_size}, lr: {lr}, alpha: {alpha}\n")

# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

# dataset 

root = 'modelnet40_normal_resampled/'
shape_names = np.loadtxt(root+"modelnet10_shape_names.txt", dtype="str")
train_data= ModelNet40Dataset(root=root, augment=True, full_dataset=full_dataset, split='train')
test_data = ModelNet40Dataset(root=root, augment=True, full_dataset=full_dataset,  split='test')


train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


num_train = len(train_data)
num_test = len(test_data)
print('Train dataset size: ', num_train)
print('Test dataset size: ', num_test)


Params: epochs: 10, batch: 2, lr: 0.001, alpha: 0.5

device: cuda
# Total clouds 10
# Total clouds 10
Train dataset size:  10
Test dataset size:  10


In [4]:
use_normal = False if dataset == "kitti" else True

# Initialize the model
model = DeepVCP(use_normal=use_normal)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

# Retrain
if retrain_path:
    print("Retrain on ", retrain_path)
    model.load_state_dict(torch.load(retrain_path))
else:
    print("No retrain")

# Define the optimizer
optim = Adam(model.parameters(), lr=lr)

# begin train 

No retrain


In [5]:
# begin train 
model.train()
loss_epoch_avg = []
for epoch in range(num_epochs):
    print(f"epoch #{epoch}")
    loss_epoch = []
    running_loss = 0.0

    for n_batch, (src, target, R_gt, t_gt, ) in enumerate(train_loader):
        start_time = time.time()
        # mini batch
        src, target, R_gt, t_gt = src.to(device), target.to(device), R_gt.to(device), t_gt.to(device)
        t_init = torch.randn((1, 3))
        R_prior = R_gt.clone()
        corruption = torch.FloatTensor(batch_size, 3, 3).uniform_(0.9, 1.1).cuda()
        R_prior = R_prior*corruption
        src_keypts, target_vcp = model(src, target, R_gt, t_init)
        # print('src_keypts shape', src_keypts.shape)
        # print('target_vcp shape', target_vcp.shape)
        # zero gradient 
        optim.zero_grad()
        loss, R_pred, t_pred = deepVCP_loss(src_keypts, target_vcp, R_gt, t_gt, alpha=0.5)


        # error metric for rigid body transformation
        r_pred = R.from_matrix(R_pred.squeeze(0).cpu().detach().numpy())
        r_pred_arr = torch.tensor(r_pred.as_euler('xyz', degrees=True)).reshape(1, 3)
        r_gt = R.from_matrix(R_gt.squeeze(0).cpu().detach().numpy())
        r_gt_arr = torch.tensor(r_gt.as_euler('xyz', degrees=True)).reshape(1, 3)
        pdist = nn.PairwiseDistance(p = 2)
        t_pred = t_pred.squeeze(-1)
        t_gt = t_gt.squeeze(-1)

        print("rotation error: ", pdist(r_pred_arr, r_gt_arr).mean())
        print("translation error: ", pdist(t_pred, t_gt).mean())

        # backward pass
        loss.backward()
        # update parameters 
        optim.step()

        running_loss += loss.item()
        loss_epoch += [loss.item()]
        print("--- %s seconds ---" % (time.time() - start_time))
        if (n_batch + 1) % 5 == 0:
            print("Epoch: [{}/{}], Batch: {}, Loss: {}".format(
                epoch, num_epochs, n_batch, loss.item()))
            running_loss = 0.0

    torch.save(model.state_dict(), "epoch_" + str(epoch) + "_model.pt")
    loss_epoch_avg += [sum(loss_epoch) / len(loss_epoch)]
    print("loss epoch", loss_epoch)
    with open("training_loss_" + str(epoch) + ".txt", "wb") as fp:   #Pickling
        pickle.dump(loss_epoch, fp)


epoch #0
Processing file: bathtub_0001
Processing file: bathtub_0002
feature extraction time:  8.620491981506348
src k id torch.Size([2, 64])
src_keypts_idx_unsqueezed:  torch.Size([2, 6, 64])
src_keypts:  torch.Size([2, 64, 6])
Grouping keypoints time:  0.028000354766845703
sample torch.Size([2, 64, 3])
B:  2
K_topk:  64
nsample:  32
num_feat:  32
get_cat_feat_src time:  0.0
src_keypts_T torch.Size([2, 3, 64])
R_init_rep  torch.Size([2, 3, 3])
tgt_pts_xyz:  torch.Size([2, 10000, 3])
ref_pts:  torch.Size([2, 10000, 3])


c:\Users\niladutt\.conda\envs\nsd\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


dist  torch.Size([2, 32, 13824])
cpk  torch.Size([2, 64, 216, 32, 3])
dist_normalize:  torch.Size([2, 32, 13824])
feat_weight_map:  torch.Size([2, 32, 32, 13824])
idx_1_mask:  torch.Size([2, 2])
idx_1_mask_flatten:  torch.Size([4])
idx_2_mask:  torch.Size([442368])
entering
tgt_deep_feat_pts torch.Size([2, 10000, 32])
idx_1_mask torch.Size([4])
idx_2_mask torch.Size([442368])


IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [4], [442368]

In [ ]:
x = [torch.randn(1,32,3), torch.randn(1,32,3)]

In [ ]:
torch.vstack(x).shape

torch.Size([2, 32, 3])

In [ ]:
z = torch.randn(1,64)

In [ ]:
(z.unsqueeze(1)).repeat(1, 6, 1).shape

torch.Size([1, 6, 64])

In [ ]:
R_gt.shape

torch.Size([2, 3, 3])

In [ ]:
R_gt.repeat(2, 1, 1).shape

torch.Size([4, 3, 3])